In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline


import torch.nn as nn

from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

import torch.nn.functional as F



In [3]:
dataset = MNIST(root='data/', download=True)
len(dataset)

In [4]:
test_dataset = MNIST(root='data/', train=False)
len(test_dataset)

In [5]:
dataset[0]

In [6]:
image, label = dataset[0]
plt.imshow(image, cmap='gray')
print('label:', label)

In [7]:
image, label = dataset[10]
plt.imshow(image, cmap='gray')
print('label:', label)

In [8]:
#transforming the PIL image into Tensors
dataset = MNIST(root='data/', 
                train=True, 
                transform=transforms.ToTensor())

In [9]:
img_tensor, label = dataset[0]
print(img_tensor.shape, label)

In [10]:
print(img_tensor[:,10:15,10:15])
print(torch.max(img_tensor), torch.min(img_tensor))

In [11]:
plt.imshow(img_tensor[0, 10:15, 10:15], cmap='gray')

In [12]:
#old one
def split_indices(n, val_pct):
    n_val= int(val_pct*n)
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

In [ ]:
#new one
from torch.utils.data import random_split

train_ds, val_ds = random_split(dataset, [50000, 10000])
len(train_ds), len(val_ds)

In [ ]:
#new one

from torch.utils.data import DataLoader

batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [14]:
#old one
train_indices, val_indices = split_indices(len(dataset), val_pct=0.2)
print(len(train_indices), len(val_indices))
print('Sample val indices: ', val_indices[:20])

In [ ]:
#new one

import torch.nn as nn

input_size = 28*28
num_classes = 10

# Logistic regression model
model = nn.Linear(input_size, num_classes)

In [16]:
#old one

#creating dataloader
batch_size = 100

#training sample and data loader
train_sampler = SubsetRandomSampler(train_indices)
train_loader = DataLoader(dataset, batch_size, sampler=train_sampler)

#validation sampler and data loader
val_sampler = SubsetRandomSampler(val_indices)
val_loader = DataLoader(dataset, batch_size, sampler=val_sampler)

In [17]:
input_size = 28*28
num_classes = 10

model = nn.Linear(input_size, num_classes)

In [18]:
print(model.weight.shape)
model.weight

In [19]:
print(model.bias.shape)
model.bias

In [20]:
for images, labels in train_loader:
    print(labels)
    print(images.shape)
    outputs = model(images)
    print(outputs)
    break

In [21]:
images.shape

In [22]:
class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, xb):
        xb = xb.reshape(-1, 784)
        out = self.linear(xb)
        return out
    
model = MnistModel()

In [23]:
model.linear

In [24]:
print(model.linear.weight.shape, model.linear.bias.shape)
list(model.parameters())


In [25]:
for images, labels in train_loader:
    print(images.shape)
    outputs = model(images)
    break

print('outputs.shape : ', outputs.shape)
print('Sample outputs :\n', outputs[:2].data)

In [26]:
outputs[:2]

In [27]:
# Apply softmax for each output row
probs = F.softmax(outputs, dim=1)

# Look at sample probabilities
print("Sample probabilities:\n", probs[:2].data)

# Add up the probabilities of an output row
print("Sum: ", torch.sum(probs[0]).item())

In [28]:
max_probs, preds = torch.max(probs, dim=1) # torch.max is non differentiable and non-continuous
print(preds)
print(max_probs)

In [29]:
labels

In [30]:
#element wise comparison
labels == preds


In [38]:
torch.sum(preds == labels)

In [39]:
torch.sum(labels == preds).item() / len(labels)

In [ ]:
#old one
def accuracy2 (l1, l2):
    return torch.sum(l1 == l2).item() / len(l1)


In [54]:
outputs[3]

In [50]:
torch.max(outputs, 1)


In [55]:
#new one
def accuracy(outputs, labels):
    preds = torch.max(outputs, 1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [56]:
accuracy(labels, preds)

In [ ]:
loss_fn = F.cross_entropy

In [ ]:
loss = loss_fn(outputs, labels)
print(loss)

In [ ]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def loss_batch(model, loss_func, xb, yb, opt=None, metric=None):
    preds = model(xb)
    loss = loss_func(preds, yb)
    
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    metric_result = None
    
    if metric is not None:
        metric_result = metric(preds, yb)
    
    return loss.item(), len(xb), metric_result
        
    

In [ ]:
def evaluate(model, loss_fn, valid_dl, metric=None):
    with torch.no_grad():
        results = [loss_batch(model, loss_fn, xb, yb, metric=metric)
                  for xb, yb in valid_dl]
        losses, nums, metrices = zip(*results)
        total = np.sum(nums)
        avg_loss = np.sum(np.multiply(losses, nums)) / total
        avg_metric = None
        if metric is not None:
            avg_metric = np.sum(np.multiply(metrices, nums))/total
    return avg_loss, total, avg_metric

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.sum(preds==labels).item() /len(preds)

In [ ]:
val_loss, total, val_acc = evaluate(model, loss_fn, val_loader, metric=accuracy)
print('Loss: {:.4f}, Accuracy: {:.4f}'.format(val_loss, val_acc))

In [ ]:
#this is not the one: watch from 2:45:01 
def fitadsf(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    optimizer = opt_func(model.parameters(), lr)
    history = [] # for recording epoch-wise results
    
    for epoch in range(epochs):
        
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)

    return history

In [ ]:
def fit(epochs, model, loss_fn, opt, train_dl, valid_dl, metric=None):
    for epoch in range(epochs):
        for xb,yb in train_dl:
            loss,_,_ = loss_batch(model, loss_fn, xb, yb, opt)
            
            result = evaluate(model, loss_fn, valid_dl, metric)
            val_loss, total, val_metric = result
            
            if metric is None:
                print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, val_loss))
                
            else:
                print('Epoch [{}/{}], Loss: {:.4f}, {}:{:.4f}'.format(epoch+1, epochs, val_loss,
                                                                     metric.__name__, val_metric))
                

In [ ]:
model = MnistModel()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
fitadsf(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)